
### Load the required libraries for translation , synonyms


In [ ]:
!pip install googletrans==4.0.0-rc1

     |████████████████████████████████| 61kB 3.5MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 1.0MB 12.4MB/s 
     |████████████████████████████████| 61kB 6.7MB/s 
     |████████████████████████████████| 71kB 7.8MB/s 
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-cp37-none-any.whl size=17417 sha256=d643e76c7b4df6245be4d620325285d32a007c7201ca415bb9fc0b3868c7cb68
  Stored in directory: /root/.cache/pip/wheels/09/32/56/fd8940f1b3c1d77c9f91b55597c52a4d4833b000a980bb0740
Successfully built googletrans


In [ ]:
!pip install spacy-wordnet

     |████████████████████████████████| 655kB 5.1MB/s 
     |████████████████████████████████| 1.4MB 11.3MB/s 
  Created wheel for spacy-wordnet: filename=spacy_wordnet-0.0.4-py2.py3-none-any.whl size=650293 sha256=0f7ad1666c230637c42189323daf18c9013889ce3150e50682a52511d0b560aa
  Stored in directory: /root/.cache/pip/wheels/25/93/1d/c86db913cd146fc9ddb26d10f56579c5d58a3e00bc8f96a3a6
  Created wheel for nltk: filename=nltk-3.3-cp37-none-any.whl size=1394470 sha256=b30315783f35e9380cf2d75b74312995bd9f195a9753f89721e2e3a40a524697
  Stored in directory: /root/.cache/pip/wheels/d1/ab/40/3bceea46922767e42986aef7606a600538ca80de6062dc266c
Successfully built spacy-wordnet nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


### Import the required libraries

In [ ]:
import torch
import torch.nn as nn
import re
import pandas as pd
import random
from spacy.lang.en import English
from datetime import datetime
import googletrans
from googletrans import Translator
import spacy

from spacy_wordnet.wordnet_annotator import WordnetAnnotator 

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

### Code to get the label cardinality


In [ ]:
import math
values = [0.34 , 0.6 , 0.7 , 0.4]



### algorithm for generating the raw dataset

*   parse the sentiment_labels.txt and create a dictionary of Phrase_id to label
*   parse the dictionary.txt and create a dictionary of sentence to phrase_id
*   parse the datasetSentences.txt and extract the string id and the comment , store this in a dictionary and also create a dataframe of comment and label



In [ ]:
counter = 0
deletecounter = 0
insertcounter = 0
translatecounter = 0

### Create a class StanfordSentimentAnalysis to encapsulate methods and attributes 

#### Following API's are supported

1. getLabelIndex(self , value)

>> Input : a float value

>> Output : an integer label

| range of float value | integer label |
| --- | --- |
| 0 <= value <= 0.2 | **0** |
| 0.2 < value <= 0.4 | **1** |
| 0.4 < value <= 0.6 | **2** |
| 0.6 < value <= 0.8 | **3** |
| 0.8 < value <= 1.0 | **4** |

2. getSentences(self)

>> This method will load the datasetSentences file and extract

>>> string id's

>>> review comments

>> store this in a dataframe

3. getPhraseIds(self):

>> This method will load the dictionary file and extract

>>> phrases 

>>> phrase ids

>> store this in a dictionary

4.  getSentimentLabels(self)

>> This method will load the sentiment_labels file and extract

>>> phrases ids 

>>> labels which are float value between 0 and 1 , including 0 and 1

>> store this in a dictionary

5.  getSentenceLabelsDF(self):

>> This method finds label for the review comment , the algorithm for the same is given below:

>>> go through the list of review comments

>>> check if the sentence exists as a phrase in the dictionary

>>>> if yes then get the phrase id and find the label for the given phrase id in labels dictionary

>>> store the review comment and the label in a data frame object

6.  getSentenceandLabels(self)

>> This is a wrapper function which call

>>> getSentimentLabels()

>>> getPhraseIds()
    
>>> getSentences()

>>> getSentenceLabelsDF()

>> The result is stored in a sentence label data frame

7.  getSpacyTokens(self , sentence):

>> This is a helper method to get spacy tokens of a sentence, a rule has been included to ignore apostrophe as a separate token.

>>> Input : Sentence

>>> Output - tokens


8.  random_deletion(self , sentence, p=0.3)

>> This method randomly deletes words from a sentence based on a probability.If the probability is less then a threshold , then drop the word.

>>> Input : Sentence

>>> Output - modified sentence

9.  random_swap(self , sentence, n=3):

>> This method randomly swaps n number of words in a sentence.

>>> Input : Sentence

>>> Output - modified sentence

10.  getReTranslatedSentence(self , sentence):

>> This method uses google translation package to translate the sentence to a random destination language.WHich is then retranslated to english.

>>> Input : Sentence

>>> Output - modified sentence

11.  removeStopWord(self, sentence):

>> This method uses spacy token attributes to check if it is a stop word

>>> If yes then drop the word

>>> Input : Sentence

>>> Output - modified sentence

12.  insertSynonymNTimes(self , input , numTimes):

>> This method is used to insert synonym for a word n number of times , this is a way to emphasise some words in the sentence.Wordnet library is used to get synonym for a word.The list of synonym is narrowed down based on domain.SInce these sentences are about movie , so the domain is set to celluloid etc.More about domain information can be found at domains - https://wndomains.fbk.eu/hierarchy.html

>>> Input : Sentence

>>> Output - modified sentence

13.  random_insertion(self , sentence, howmanytimes = 3): 

>> This is a wrapper method which call the removestop word and insertSynonymNTimes.

>>> Input : Sentence

>>> Output - modified sentence



In [ ]:
class StanfordSentimentAnalysis:
  def __init__(self):

    self.sentencesDF = pd.DataFrame()
    self.phraseIdDict = {}
    self.labelsDict = {}
    self.sentencesLabelDF = pd.DataFrame()

  def getLabelIndex(self , value):

    num_of_bins= 5
    bin_width = 0.2
    bins = [0] * num_of_bins
    #print(bins)
    bin_num = -1

    if bin_width*0 <= value <= bin_width :
      bin_num = 0
    elif bin_width*1 < value <= bin_width*2 :
      bin_num = 1
    elif bin_width*2 < value <= bin_width*3 :
      bin_num = 2
    elif bin_width*3 < value <= bin_width*4 :
      bin_num = 3
    elif bin_width*4 < value <= bin_width*5 :
      bin_num = 4
    #print(value , bin_num)
    
    return bin_num  
  
  
  def getSentences(self):

    with open('/content/datasetSentences.txt','r') as sent:
      idc = 0
      sc = 0
      indexes = []
      sentiments = []
      lines = []
      for lin in sent.readlines():
    
        end = 0
        if idc == 0:
          idc += 1
          sc += 1
          continue
        if(re.search(r'\d+', lin)):
          index = re.search(r'\d+', lin)
          end = index.end()
          indexes.append(index.group())
          idc +=1
        if(re.search(r'\D+', lin)):
          sentence = lin[end+1 : ].strip()
          #print(sentence)
          sentiments.append(sentence)
          sc += 1
        assert idc == sc      
        lines.append(lin)

      self.sentencesDF["sid"] = indexes
      self.sentencesDF["comment"] = sentiments


  def getPhraseIds(self):

    with open('/content/dictionary.txt','r') as dif:
      for lin in dif.readlines():
        tok = re.split(r'\|',lin)
        self.phraseIdDict[tok[0]] = int(tok[1].strip())


  def getSentimentLabels(self):

    with open('/content/sentiment_labels.txt','r') as lbf:
      i = -1
      for lin in lbf.readlines():
        i += 1
        if i == 0:
          continue
    
        tok = re.split(r'\|',lin)          
        self.labelsDict[int((tok[0]))] = float(tok[1].strip())

  def getSentenceLabelsDF(self):

    sentiments = []
    labels = []
    i = 0
    for cmnt in self.sentencesDF.comment :

      if cmnt in self.phraseIdDict:
        phraseId = self.phraseIdDict[cmnt]
        label = self.labelsDict[phraseId]
        sentiments.append(cmnt)
        labels.append(self.getLabelIndex(label))
        i += 1

    assert i > 11000
    
    self.sentencesLabelDF['sentence'] = sentiments
    self.sentencesLabelDF['label'] = labels

    return self.sentencesLabelDF

  def getSentenceandLabels(self)        :

    self.getSentimentLabels()
    self.getPhraseIds()
    self.getSentences()

    return self.getSentenceLabelsDF()


  def getSpacyTokens(self , sentence):

    nlp = English()
    nlp.tokenizer.rules = {key: value for key, value in nlp.tokenizer.rules.items() if "'" not in key and "’" not in key and "‘" not in key}

    #  "nlp" Object is used to create documents with linguistic annotations.
    my_doc = nlp(sentence)

    # Create list of word tokens
    return [token.text for token in my_doc]


  def random_deletion(self , sentence, p=0.3): 
    global deletecounter
    start_time = datetime.now()
    words = self.getSpacyTokens(sentence)

    if len(words) == 1: # return if single word
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    newwords = ''
    if len(remaining) == 0: # if not left, sample a random word
        newwords = [random.choice(words)] 
    else:
        newwords = remaining

    end_time = datetime.now()

    time_diff = (end_time - start_time)

    execution_time = time_diff.total_seconds() * 1000

    deletecounter = deletecounter + 1
    print( deletecounter , execution_time)

    return ' '.join(newwords)

  def random_swap(self , sentence, n=3): 
    global counter
    start_time = datetime.now()

    words = self.getSpacyTokens(sentence)

    length = range(len(words)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        #print(idx1, idx2)
        words[idx1], words[idx2] = words[idx2], words[idx1] 

    end_time = datetime.now()

    time_diff = (end_time - start_time)

    execution_time = time_diff.total_seconds() * 1000

    counter = counter + 1
    print( counter , execution_time)

    return ' '.join(words)

  def getReTranslatedSentence(self , sentence):

    global translatecounter

    start_time = datetime.now()

    translator = Translator()

    available_langs = list(googletrans.LANGUAGES.keys()) 
    trans_lang = random.choice(available_langs) 
    #print(f"Translating to {googletrans.LANGUAGES[trans_lang]}")
    
    #print(sentence)
    translations = translator.translate(sentence, dest=trans_lang).text 

    translations_en_random = translator.translate(translations, src=trans_lang, dest='en').text 

    #print('back-',translations_en_random)

    end_time = datetime.now()

    time_diff = (end_time - start_time)

    execution_time = time_diff.total_seconds() * 1000

    translatecounter = translatecounter + 1
    print('tlt ' ,translatecounter , execution_time)

    return translations_en_random

  def removeStopWord(self, sentence):
    nlp = spacy.load('en', parser=False, entity=False)  
    nlp.tokenizer.rules = {key: value for key, value in nlp.tokenizer.rules.items() if "'" not in key and "’" not in key and "‘" not in key}
    return ' '.join([token.lemma_ for token in nlp(sentence) if not token.is_stop])

  def insertSynonymNTimes(self , input , numTimes):

    # Load an spacy model (supported models are "es" and "en") 
    nlp = spacy.load('en')
    nlp.add_pipe(WordnetAnnotator(nlp.lang), after='tagger')
    token = nlp('prices')[0]

    # wordnet object link spacy token with nltk wordnet interface by giving acces to
    # synsets and lemmas 
    token._.wordnet.synsets()
    token._.wordnet.lemmas()

    # And automatically tags with wordnet domains
    token._.wordnet.wordnet_domains()

    # Imagine we want to enrich the following sentence with synonyms
    sentence = nlp(input)

    # spaCy WordNet lets you find synonyms by domain of interest
    # for example economy
    #movie_domains = ['banking', 'finance','cinema']
    movie_domains = ['film', 'music' , 'celluloid' ,'play', 'sport', 'history' , 'literature', 'banking' , 'finance' ,'cinema']
    enriched_sentence = []
    

    # For each token in the sentence
    for token in sentence:
      #print('idx-',token.i)
      # We get those synsets within the desired domains
      synsets = token._.wordnet.wordnet_synsets_for_domain(movie_domains)
      #synsets = token._.wordnet.synsets()
      if synsets:
          lemmas_for_synset = []
          for s in synsets:
            # If we found a synset in the economy domains
            # we get the variants and add them to the enriched sentence
            #print(s.lemma_names())
            lemmas_for_synset.extend(s.lemma_names())
            if (len(lemmas_for_synset) > 1):
              #print('lfors-',lemmas_for_synset[1:])
              #enriched_sentence.append('({})'.format('|'.join(set(lemmas_for_synset))))
              ntimes = (random.choice(lemmas_for_synset[1:]) + ' ')*numTimes
              #print(ntimes)
              enriched_sentence.append(ntimes)
            else:
              enriched_sentence.append(token.text)
      else:
        enriched_sentence.append(token.text)
        #mod_sent.append(token.text)

    # Let's see our enriched sentence
    return ' '.join(enriched_sentence)
          

  def random_insertion(self , sentence, howmanytimes = 3): 

    global insertcounter
    start_time = datetime.now()

    newSentence = self.removeStopWord(sentence) 
    synmsentence =  self.insertSynonymNTimes(newSentence,howmanytimes)

    end_time = datetime.now()

    time_diff = (end_time - start_time)

    execution_time = time_diff.total_seconds() * 1000
    
    insertcounter = insertcounter + 1
    print('ins', insertcounter , execution_time)

    return synmsentence 

### Instantiate the StanfordSentimentAnalysis class and get the comments and label dataframe

In [ ]:
sa = StanfordSentimentAnalysis()
sldf = sa.getSentenceandLabels()

#### save the sentiment analysis dataframe

In [ ]:
sldf.to_csv (r'/content/sentanlysdf.csv', index = False, header=True)

In [ ]:
len(sldf)

11286

#### Test random insertion augmentation method

In [ ]:
sa.random_insertion (sldf.sentence[11283],3)


ins 1 4507.451


"' ve view view view  see see see  hippy hippy hippy  - play play play  round round round  round round round  - yuppie plot , ' s enthusiastic charm Fire flame flame flame  do do do  do do do  make make make  make make make  induce induce induce  produce produce produce  get get get  produce produce produce  pull_in pull_in pull_in  bring_in bring_in bring_in  make make make  create create create  take_in take_in take_in  create create create  make make make  realize realize realize  formula fresh ."

#### Test random swap augmentation method

In [ ]:
sa.random_swap(sldf.sentence[1123])

1 326.322


"Writer\\/director Joe Carnahan moves s grimy crime drama clunky cliches manual of precinct a , but it ' fast enough to cover its is dialogue and lapses in logic ."

#### Test random deletion augmentation method

In [ ]:
sa.random_deletion(sldf.sentence[11283])

1 336.189


"' the hippie - turned - plot , there s an enthusiastic charm in Fire that makes the formula again"

### utility function to call different augmentation function

#### Input :

1.   data frame object
2.   augmentation method 

#### Output:

1.   augmented data frame object







In [ ]:
def augmentStanfordSentimentAnalysisData(sentAnlysDF , method):

  swapSentence = sentAnlysDF.sentence.apply(method)
  swapLabel = sentAnlysDF.label

  swapDF = pd.concat([swapSentence, swapLabel], axis=1)

  return swapDF
  


### Apply the random swap method on each sentence in the dataset

In [ ]:
augDF = augmentStanfordSentimentAnalysisData(sldf , sa.random_swap)
len(augDF)

In [ ]:
augDF.to_csv (r'/content/augwrapdf.csv', index = False, header=True)

### Concatenate the augmented sentences to the original dataset

In [ ]:
orig_swapDF =  pd.concat([sldf , augDF],ignore_index=True)

In [ ]:
orig_swapDF.to_csv (r'/content/orig_wrapdf.csv', index = False, header=True)


### Apply the random deletion method to each sentence in the dataset

In [ ]:
augDelDF = augmentStanfordSentimentAnalysisData(sldf , sa.random_deletion)
len(augDelDF)

In [ ]:
augDelDF.to_csv (r'/content/augdeldf.csv', index = False, header=True)

In [ ]:
sldf[:10]

,sentence,label
0,The Rock is destined to be the 21st Century 's...,3
1,The gorgeously elaborate continuation of `` Th...,4
2,Effective but too-tepid biopic,2
3,If you sometimes like to go to the movies to h...,3
4,"Emerges as something rare , an issue movie tha...",4
5,The film provides some great insight into the ...,2
6,Offers that rare combination of entertainment ...,4
7,Perhaps no picture ever made has more literall...,3
8,Steers turns in a snappy screenplay that curls...,3
9,But he somehow pulls it off .,3


In [ ]:
len(orig_swapDF)

22572

### Concatenate the modified sentence to the dataset which has original + words swapped in the sentence


In [ ]:
orig_swap_delDF =  pd.concat([orig_swapDF , augDelDF],ignore_index=True)

In [ ]:
len(orig_swap_delDF)

33858

In [ ]:
orig_swap_delDF.to_csv (r'/content/orig_wrap_deldf.csv', index = False, header=True)

### Apply the google retranslation method to each sentence in the original dataset

In [ ]:
augtrnsltDF = augmentStanfordSentimentAnalysisData(sldf , sa.getReTranslatedSentence)
len(augtrnsltDF)

In [ ]:
augtrnsltDF.to_csv (r'/content/aug_tltdf.csv', index = False, header=True)

### Concatenate the translated sentences to original + swapped + deleted dataset

In [ ]:
orig_swap_del_tltDF =  pd.concat([orig_swap_delDF , augtrnsltDF],ignore_index=True)

In [ ]:
orig_swap_del_tltDF.to_csv (r'/content/orig_wrap_del_tltdf.csv', index = False, header=True)

### Apply the random insertion method to each sentence in the original dataset.This was the most expensive operation.It took an avg of 2500 milliseconds to do random insertion.

In [ ]:
auginsertDF = augmentStanfordSentimentAnalysisData(sldf , sa.random_insertion)
len(auginsertDF)

ins 1 2039.52
ins 2 2132.431
ins 3 2142.6620000000003
ins 4 1884.3229999999999
ins 5 2089.708
ins 6 1923.519
ins 7 2052.1240000000003
ins 8 1879.496
ins 9 2130.5930000000003
ins 10 1908.346


11286

In [ ]:
auginsertDF.to_csv (r'/content/aug_insdf.csv', index = False, header=True)

,sentence,label
0,The rock is in the destiny of the 21st century...,3
1,The elaborate continuation of `` The Lord of t...,4
2,Effective but tepid biopic,2
3,"Sometimes if you want to travel to the movies,...",3
4,Something comes out and it observes a fair and...,4
...,...,...
11281,Repeat real.,0
11282,Not surprised.,1
11283,We have seen the hippy-turned-jopy plot before...,3
11284,Her lovers face poetry with poetry like '' and...,0


Concatenate the randomly inserted words in the sentence to original + swapped + deleted + translated dataset

In [ ]:
orig_swap_del_tlt_insDF =  pd.concat([orig_swap_del_tltDF , auginsertDF],ignore_index=True)

In [ ]:
orig_swap_del_tlt_insDF.to_csv (r'/content/orig_swap_del_tlt_insDF.csv', index = False, header=True)

In [ ]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    2.2.4                         
Location         /usr/local/lib/python3.7/dist-packages/spacy
Platform         Linux-5.4.109+-x86_64-with-Ubuntu-18.04-bionic
Python version   3.7.10                        
Models           en                            

